In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
import os

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("C:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection\\tests")
THIS_DIR = Path(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests")
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [3]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
# from skopt.space import Real
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection._search import bayesian_search_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog

In [4]:
import os
import re
import sys
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._search import _bayesian_search_optuna_multiseries
from skforecast.model_selection._search import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection._search import _evaluate_grid_hyperparameters
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

import re
import pytest
import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection._search import _bayesian_search_optuna
from skforecast.model_selection._split import TimeSeriesFold
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod
from sklearn.linear_model import LinearRegression
from skforecast.model_selection._validation import _backtesting_forecaster

In [37]:
window_features = RollingFeatures(
    stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
    regressor=LinearRegression(), steps=4, lags=[], window_features=window_features
)

In [29]:
lags, lags_names, max_lag = (None, None, None)

In [39]:
[
    feature for feature in [1, 2, 3] if feature in []
]

[]

In [27]:
window_features = RollingFeatures(
    stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
    regressor=LinearRegression(), steps=4, lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_squared_error',
                                    verbose    = False
                                )

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
metric.to_numpy()

array([[0.11337407]])

In [29]:
backtest_predictions.to_numpy().flatten()

array([0.31478125, 0.45940314, 0.85744121, 0.67976412, 0.38085263,
       0.56357911, 0.35567212, 0.4493073 , 0.35493806, 0.53589892,
       0.73060039, 0.60025524])

In [9]:
results.to_dict(orient='list')

{'lags': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'lags_label': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'params': [{'alpha': 1},
  {'alpha': 1},
  {'alpha': 0.1},
  {'alpha': 0.1},
  {'alpha': 0.01},
  {'alpha': 0.01}],
 'mean_squared_error': [0.07424336814199613,
  0.07576112822191301,
  0.07677225532446876,
  0.08046370393440734,
  0.08411584414110274,
  0.08911485459058603],
 'alpha': [1.0, 1.0, 0.1, 0.1, 0.01, 0.01]}

In [30]:
import numpy as np
import pandas as pd
from skforecast.sarimax import Sarimax
from skforecast.recursive import ForecasterSarimax
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._search import random_search_sarimax

# Fixtures
from skforecast.recursive.tests.tests_forecaster_sarimax.fixtures_forecaster_sarimax import y_datetime


In [32]:
forecaster = ForecasterSarimax(
                     regressor = Sarimax(order=(3, 2, 0), maxiter=1000, method='cg', disp=False)
                 )
    
cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = len(y_datetime) - 12,
            refit                 = False,
            fixed_train_size      = False,
            gap                   = 0,
            allow_incomplete_fold = True
        )
cv = OneStepAheadFold(
            initial_train_size    = len(y_datetime) - 12,
            return_all_indexes    = False,
        )

param_distributions = {'order': [(2, 2, 0), (3, 2, 0)]}

results = random_search_sarimax(
                forecaster          = forecaster,
                y                   = y_datetime,
                cv                  = cv,
                param_distributions = param_distributions,
                metric              = 'mean_absolute_error',
                n_iter              = 2,
                random_state        = 123,
                return_best         = False,
                verbose             = False
            )

Number of models compared: 2.


params grid:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: `cv` must be a TimeSeriesFold object. Got OneStepAheadFold.

In [34]:
cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = 10,
            refit                 = False,
            fixed_train_size      = False,
            gap                   = 0,
            allow_incomplete_fold = True
        )

In [36]:
current_params = vars(cv)
current_params

params = {'steps': 53}

In [37]:
{'cv_name': type(cv).__name__, **current_params, **params}

{'cv_name': 'TimeSeriesFold',
 'steps': 53,
 'initial_train_size': 10,
 'window_size': None,
 'differentiation': None,
 'refit': False,
 'fixed_train_size': False,
 'gap': 0,
 'skip_folds': None,
 'allow_incomplete_fold': True,
 'return_all_indexes': False,
 'verbose': True}

In [51]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = 70,
        window_size           = None,
        differentiation       = None,
        refit                 = False,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
        verbose               = False
    )
folds = cv.split(X=y, as_pandas=True)
folds

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_split.py:791: UserWarning: Last window cannot be calculated because `window_size` is None.
  externally_fitted = False


,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,70,None,None,70,80,70,80,True
1,1,0,70,None,None,80,90,80,90,False
2,2,0,70,None,None,90,100,90,100,False


In [60]:
expected = pd.DataFrame(
        {'fold': [0, 1, 2],
         'train_start': [0, 0, 0],
         'train_end': [70, 70, 70],
         'last_window_start': [67, 77, 87],
         'last_window_end': [70, 80, 90],
         'test_start': [70, 80, 90],
         'test_end': [80, 90, 100],
         'test_start_with_gap': [70, 80, 90],
         'test_end_with_gap': [80, 90, 100],
         'fit_forecaster': [True, False, False]}
    )
expected['last_window_start'] = [None, None, None]
expected

,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,70,None,70,70,80,70,80,True
1,1,0,70,None,80,80,90,80,90,False
2,2,0,70,None,90,90,100,90,100,False


In [63]:
expected.dtypes

fold                    int64
train_start             int64
train_end               int64
last_window_start      object
last_window_end         int64
test_start              int64
test_end                int64
test_start_with_gap     int64
test_end_with_gap       int64
fit_forecaster           bool
dtype: object

In [62]:
folds.dtypes

fold                    int64
train_start             int64
train_end               int64
last_window_start      object
last_window_end        object
test_start              int64
test_end                int64
test_start_with_gap     int64
test_end_with_gap       int64
fit_forecaster           bool
dtype: object

In [44]:
folds.to_dict(orient='list')

{'fold': [0, 1, 2],
 'train_start': [0, 0, 0],
 'train_end': [70, 70, 70],
 'last_window_start': [67, 77, 87],
 'last_window_end': [70, 80, 90],
 'test_start': [70, 80, 90],
 'test_end': [80, 90, 100],
 'test_start_with_gap': [70, 80, 90],
 'test_end_with_gap': [80, 90, 100],
 'fit_forecaster': [True, False, False]}

In [56]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = 70,
        window_size           = None,
        differentiation       = None,
        refit                 = False,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = True,
        verbose               = False
    )
folds_2 = cv.split(X=y, as_pandas=True)
folds_2

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_split.py:791: UserWarning: Last window cannot be calculated because `window_size` is None.
  externally_fitted = False


,fold,train_index,last_window_index,test_index,test_index_with_gap,fit_forecaster
0,0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(70, 71, 72, 73, 74, 75, 76, 77, 78, 79)","(70, 71, 72, 73, 74, 75, 76, 77, 78, 79)",True
1,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(80, 81, 82, 83, 84, 85, 86, 87, 88, 89)","(80, 81, 82, 83, 84, 85, 86, 87, 88, 89)",False
2,2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[None, None]","(90, 91, 92, 93, 94, 95, 96, 97, 98, 99)","(90, 91, 92, 93, 94, 95, 96, 97, 98, 99)",False


In [58]:
folds_2.dtypes

fold                    int64
train_index            object
last_window_index      object
test_index             object
test_index_with_gap    object
fit_forecaster           bool
dtype: object

In [57]:
folds_2.to_dict(orient='list')

{'fold': [0, 1, 2],
 'train_index': [range(0, 70), range(0, 70), range(0, 70)],
 'last_window_index': [[None, None], [None, None], [None, None]],
 'test_index': [range(70, 80), range(80, 90), range(90, 100)],
 'test_index_with_gap': [range(70, 80), range(80, 90), range(90, 100)],
 'fit_forecaster': [True, False, False]}

In [64]:
from skforecast.preprocessing import QuantileBinner

In [70]:
X = np.arange(1000)
binner = QuantileBinner(
    n_bins=10,
    method='linear',
    subsample=10,
    dtype=np.float64,
    random_state=789654,
)

binner.fit(X.reshape(-1, 1))


In [71]:
binner.bin_edges_

array([ 21. , 159.6, 283.8, 388.7, 429.8, 578. , 725. , 731.6, 734.2,
       739.4, 743. ])

In [72]:
binner.n_bins_

10

In [73]:
binner.intervals_

{0.0: (21.0, 159.6),
 1.0: (159.6, 283.8),
 2.0: (283.8, 388.7),
 3.0: (388.7, 429.8),
 4.0: (429.8, 578.0),
 5.0: (578.0, 725.0),
 6.0: (725.0, 731.6),
 7.0: (731.6, 734.2),
 8.0: (734.2, 739.4),
 9.0: (739.4, 743.0)}

In [4]:
import re
import pytest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.feature_selection import select_features
from skforecast.feature_selection import select_features_multiseries
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.feature_selection.tests.tests_feature_selection.fixtures_feature_selection import y_feature_selection as y
from skforecast.feature_selection.tests.tests_feature_selection.fixtures_feature_selection import exog_feature_selection as exog
from skforecast.feature_selection.tests.tests_feature_selection.fixtures_feature_selection import series
from skforecast.feature_selection.tests.tests_feature_selection.fixtures_feature_selection import exog_multiseries as exog

In [5]:
roll_features = RollingFeatures(
                        stats=['mean', 'std'],
                        window_sizes=[3, 5],
                    )
forecaster = ForecasterDirectMultiVariate(
                     regressor = LinearRegression(),
                     level     = 'l1',
                     steps     = 3,
                     lags      = {'l1': [], 'l2': 5},
                     window_features = roll_features
                 )
print(forecaster.lags)
print(forecaster.lags_names)
print(forecaster.max_lag)

{'l1': None, 'l2': array([1, 2, 3, 4, 5])}
{'l1': None, 'l2': ['l2_lag_1', 'l2_lag_2', 'l2_lag_3', 'l2_lag_4', 'l2_lag_5']}
5


In [6]:
forecaster.fit(series=series)
print(forecaster.lags_)
print(forecaster.lags_names)

{'l1': None, 'l2': array([1, 2, 3, 4, 5])}
{'l1': None, 'l2': ['l2_lag_1', 'l2_lag_2', 'l2_lag_3', 'l2_lag_4', 'l2_lag_5']}


In [11]:
forecaster = ForecasterDirectMultiVariate(
                    regressor = LinearRegression(),
                    level     = 'l1',
                    steps     = 3,
                    lags      = 5
                )
selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

selected_lags, selected_window_features, selected_exog = select_features_multiseries(
    selector    = selector,
    forecaster  = forecaster,
    series      = series,
    exog        = exog,
    select_only = 'autoreg',
    verbose     = False,
)

{'l1': ['l1_lag_1'], 'l2': ['l2_lag_1', 'l2_lag_4']}


In [27]:
selected_lags

{'l1': [], 'l2': [1, 4, 5]}

In [28]:
forecaster = ForecasterDirectMultiVariate(
                    regressor = LinearRegression(),
                    level     = 'l1',
                    steps     = 3,
                    lags      = selected_lags
                )
print(forecaster.lags)
print(forecaster.lags_names)
print(forecaster.max_lag)

{'l1': None, 'l2': array([1, 4, 5])}
{'l1': None, 'l2': ['l2_lag_1', 'l2_lag_4', 'l2_lag_5']}
5


In [29]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.utils import transform_numpy
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict

THIS_DIR = Path(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\tests\tests_forecaster_recursive_multiseries")
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [50]:
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[5, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[6])

forecaster = ForecasterRecursiveMultiSeries(
    regressor          = LGBMRegressor(
        n_estimators=10, random_state=123, verbose=-1, max_depth=5
    ),
    lags               = [1, 5],
    window_features    = [rolling, rolling_2],
    encoding           = 'ordinal',
    dropna_from_series = False,
    # transformer_series = StandardScaler(),
    # transformer_exog   = StandardScaler(),
    differentiation    = None
)
forecaster.fit(
    series=series_dict_train, exog=exog_dict_train, suppress_warnings=True
)
steps = 5
levels = ['id_1000', 'id_1001', 'id_1003', 'id_1004', 'id_1005']
last_window = pd.DataFrame(
    {k: v for k, v in forecaster.last_window_.items() if k in levels}
)
last_window['id_1005'] = last_window['id_1004']

X_predict = forecaster.create_predict_X(
    steps=steps, levels=levels, last_window=last_window, exog=exog_dict_test,
suppress_warnings=True
)

results = np.full(
    shape=(steps, len(levels)), fill_value=np.nan, order='F', dtype=float
)
for i, level in enumerate(levels):
    results[:, i] = transform_numpy(
        array             = forecaster.regressor.predict(X_predict[level]),
        transformer       = forecaster.transformer_series_.get(level, forecaster.transformer_series_['_unknown_level']),
        fit               = False,
        inverse_transform = True
    )

expected = forecaster.predict(
    steps=steps, levels=levels, last_window=last_window, exog=exog_dict_test,
    suppress_warnings=True
)

[[1409.251091   2160.         1994.50059509 8513.00302124 8513.00302124]
 [1344.75059509 2224.00019836 1830.8717804  8622.00201416 8622.00201416]
 [1310.00039673 2334.00049591 1776.25099182 9045.00198364 9045.00198364]
 [1297.00089264 2559.00059509 1644.001091   8733.00201416 8733.00201416]
 [1103.50039673 2911.         1943.50058746 6952.         6952.        ]
 [ 871.50019836 2903.         3658.36999512 5964.         5964.        ]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]]
[1409.251091   1344.75059509 1310.00039673 1297.00089264 1103.50039673
  871.50019836 2019.50909641 2272.41996969 2425.45554932 2559.55489734
 2538.9266253 ]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [57]:
X_predict = forecaster.create_predict_X(
    steps=steps, levels=levels, last_window=last_window, exog=exog_dict_test,
suppress_warnings=True
)

[[1409.251091   2160.         1994.50059509 8513.00302124 8513.00302124]
 [1344.75059509 2224.00019836 1830.8717804  8622.00201416 8622.00201416]
 [1310.00039673 2334.00049591 1776.25099182 9045.00198364 9045.00198364]
 [1297.00089264 2559.00059509 1644.001091   8733.00201416 8733.00201416]
 [1103.50039673 2911.         1943.50058746 6952.         6952.        ]
 [ 871.50019836 2903.         3658.36999512 5964.         5964.        ]]
[[ 1185.35049591  1297.00089264  7336.00357056]
 [ 2586.20025787  2559.00059509 15091.00128937]
 [ 2170.59888916  1830.8717804  12847.49504089]
 [ 7863.20120239  8622.00201416 47829.0090332 ]
 [ 7863.20120239  8622.00201416 47829.0090332 ]]
creeate_X 1 [1409.251091   1344.75059509 1310.00039673 1297.00089264 1103.50039673
  871.50019836]
creeate_X 2 [1185.35049591 1297.00089264 7336.00357056]
creeate_X 1 [2160.         2224.00019836 2334.00049591 2559.00059509 2911.
 2903.        ]
creeate_X 2 [ 2586.20025787  2559.00059509 15091.00128937]
creeate_X 1 [19

In [53]:
expected = forecaster.predict(
    steps=steps, levels=levels, last_window=last_window, exog=exog_dict_test,
    suppress_warnings=True
)

[[1409.251091   2160.         1994.50059509 8513.00302124 8513.00302124]
 [1344.75059509 2224.00019836 1830.8717804  8622.00201416 8622.00201416]
 [1310.00039673 2334.00049591 1776.25099182 9045.00198364 9045.00198364]
 [1297.00089264 2559.00059509 1644.001091   8733.00201416 8733.00201416]
 [1103.50039673 2911.         1943.50058746 6952.         6952.        ]
 [ 871.50019836 2903.         3658.36999512 5964.         5964.        ]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]
 [          nan           nan           nan           nan           nan]]


In [41]:
expected

,id_1000,id_1001,id_1003,id_1004,id_1005
2016-08-01,2019.509096,2657.804323,2699.058820,5472.602561,4777.114168
2016-08-02,2272.419970,2760.205171,2624.827488,5312.776507,4524.994971
2016-08-03,2425.455549,2718.031169,2569.964632,5312.776507,4524.994971
2016-08-04,2559.554897,3386.623523,2760.647885,5003.433006,3807.926740
2016-08-05,2538.926625,3886.054469,3457.976660,4585.550954,3594.938154


In [71]:
from skforecast.utils import exog_to_direct

exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                    'exog_2': np.arange(1000, 1010, dtype=float)})

exog = exog.iloc[5:,]
exog_to_direct(exog, steps=4)[0]

,exog_1_step_1,exog_2_step_1,exog_1_step_2,exog_2_step_2,exog_1_step_3,exog_2_step_3,exog_1_step_4,exog_2_step_4
8,105.0,1005.0,106.0,1006.0,107.0,1007.0,108.0,1008.0
9,106.0,1006.0,107.0,1007.0,108.0,1008.0,109.0,1009.0


In [65]:
print(np.arange(10)[5 + (3 - 1):])
exog = pd.DataFrame({'exog_1': np.arange(105, 110, dtype=float),
                     'exog_2': np.arange(1005, 1010, dtype=float)})

exog_to_direct(exog, steps=3)[0]

[7 8 9]


,exog_1_step_1,exog_2_step_1,exog_1_step_2,exog_2_step_2,exog_1_step_3,exog_2_step_3
2,105.0,1005.0,106.0,1006.0,107.0,1007.0
3,106.0,1006.0,107.0,1007.0,108.0,1008.0
4,107.0,1007.0,108.0,1008.0,109.0,1009.0
